# Finding differential test cases

The goal of this notebook is to identify and extract all differential test cases and store the functions that are used as oracles. These functions can then be checked for changes whenever developers make a new commit that the tool's user might want to know about.

## Imports

In [2]:
import ast
import sys

## Import source code and create Abstract Syntax Tree


An abstract syntax tree represents source code as a tree, where each node is an element of the source code, e.g. a function definition, function call, if-else statement, while-loop, variable assignment etc. It abstracts away e.g. grouping parentheses, because these groupings are implicitly defined in the tree's structure.

<img style="float: right;" src="images/abstract_syntax_tree.png" width="450">

This code produces the following tree:
```
while b ≠ 0
  if a > b
    a := a − b
  else
    b := b − a
return a
```

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<div style="text-align: right"> <i> Source https://en.wikipedia.org/wiki/Abstract_syntax_tree </i> </div>

In [8]:
# Set tensorflow root folder
tensorflow_root = "A:/BachelorThesis/DLL_Testing_Tool/DL_Libraries/Tensorflow/tensorflow-master/tensorflow/python/" 

# set python source file
source = open(tensorflow_root + "kernel_tests/array_ops/gather_op_test.py")

# generate abstract syntax tree
tree = ast.parse(source.read())
nodes = ast.walk(ast.parse(tree))

# print names of all defined functions in the source file
for node in nodes:
    if isinstance(node, ast.FunctionDef):
        print(node.name)

_to_str_elements
_buildParams
testScalar1D
testScalar2D
testSimpleTwoD32
testHigherRank
testHigherRankGradientTape
testString
testUInt32AndUInt64
testUnknownIndices
testUnknownAxis
testBadIndicesType
testBadIndicesCPU
_disabledTestBadIndicesGPU
testBadAxis
testEmptySlices
testBatchDims
testBatchDimsMatchesPythonBatching
_batchNumpyGather
testGatherRefVariable
testGatherResourceVariable


## Identify assert function calls and check if the arguments are differential test oracles

In [9]:
for node in nodes:
    # if the node is a function call
    if isinstance(node, ast.Call):
        
        # if the function is an assertAllClose:
        if hasattr(node.func, "attr"):
            if node.func.attr == "assertAllClose":
                
                # if the function has arguments
                if hasattr(node, "args"):
                    # for each argument of the function
                    for arg in node.args:
                        print(node.args.name)
                        
                        #if isinstance(arg,ast.Num):
                           # fields = vars(arg) 